### Imports & constants

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from random import sample
import cv2

from scipy.ndimage import zoom

In [2]:
KANAVATI_PATH = os.path.join(os.curdir, 'datasets', 'Kanavati')
OUTPUT_PATH = os.path.join(os.curdir, 'output', 'Kanavati.npz')

### Data processing

In [3]:
imgs_s = [] # np.array with sagittal MIPs
imgs_f = [] # np.array with frontal MIPs
v_levels = [] # np.array with labels in mm
ids = [] # np.array with CTs identifiers
thicks = [] # np.array with CTs slices' thicknesses

In [4]:
data = np.load(os.path.join(KANAVATI_PATH, 'l3_dataset.npz'), allow_pickle=True)

In [5]:
# following authors' convention to decode label
names = data['names'] # examination names
y = np.zeros_like(names, dtype=np.float32) # array of shape of ex_names
y_data = data['ydata']
for _, v in y_data.item().items():
        y += v
y /= len(y_data.item()) # diving by 2

In [ ]:
n_images = int(data['num_images'])
data_spacings = data['spacings']
data_images_s = data['images_s']
data_images_f = data['images_f']
data_names = data['names']

for i in range(n_images): # rescaling to 1mm x 1mm
    spacings = data_spacings[i]
    thicks.append(spacings[2])
    
    img_s = data_images_s[i]
    img_s = zoom(img_s, [spacings[2], spacings[0]])
    imgs_s.append(img_s)
    
    img_f = data_images_f[i]
    img_f = zoom(img_f, [spacings[2], spacings[0]])
    imgs_f.append(img_f)
    
    v_levels.append(
        [-1, int(min(np.round(y[i]*spacings[2]), img_s.shape[0]))]) # [T12 level, L3 level]
    
    ids.append(data_names[i])

In [7]:
# exporting to .npz
imgs_s = np.asarray(imgs_s)
imgs_f = np.asarray(imgs_f)
v_levels = np.asarray(v_levels)
ids = np.asarray(ids)
thicks = np.asarray(thicks)
n_mips = imgs_s.shape[0]

np.savez_compressed(OUTPUT_PATH, imgs_s=imgs_s, imgs_f=imgs_f,
                    v_levels=v_levels, ids=ids, thicks=thicks, n_mips=n_mips)

/home/dittohead/miniconda3/envs/thesis/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
